In [139]:
####################################################################
# VERSION 07 - 2021/07/06
####################################################################
# most used libs
from math import *
import numpy as np
import scipy as sc

####################################################################
# Used if running this python code outside MOOC test platform
if not 'OfflineMode' in globals().keys():
  OfflineMode = not 'anonymous_student_id' in globals().keys()
if OfflineMode: 
  anonymous_student_id = 'student'

####################################################################
# print output if running offline
def OfflinePrint( outtxt='' ):
  if OfflineMode: print( outtxt )

####################################################################
# init numpy random generator
def InitRandomGenerator( problem_id = '' ):
    maxint = np.iinfo(np.int32).max
    rnd_seed = hash( str( anonymous_student_id ) \
                   + str(problem_id) ) % maxint
    np.random.seed( rnd_seed )

####################################################################
def Disturb( val, dig, dist ):
  ss = +1.0 if np.random.uniform() >= 0.5 else -1.0
  rr = np.random.uniform() * ( dist - 0.1 )
  val = val + ss * val * ( rr + 0.1 )
  return round( val, dig ) 

####################################################################
# Uses the random number generator of numpy
# This function should be used instead of random.choice()
def RandomPick( lst ):
  return lst[ np.random.randint( 0, len( lst ) ) ]

####################################################################
# Creates num_entries-1 invalid answers such that
#
#  This function generates the global variables associated with
#  $a_answer_0,..., $a_answer_3 and
#  $a_value_0,..., $a_value_3 and
#
#  <multiplechoiceresponse>
#    <choicegroup type="MultipleChoice">
#      <choice correct="$a_answer_0" > $a_value_0 </choice>
#      <choice correct="$a_answer_1" > $a_value_1 </choice>
#      <choice correct="$a_answer_2" > $a_value_2 </choice>
#      <choice correct="$a_answer_3" > $a_value_3 </choice>
#    </choicegroup>
#  </multiplechoiceresponse>
#
# prefix - the '*' used in the variables *_answer_0 and *_value_0
# value - correct answer
# ndigs - digits after the comma
# suffix - usually the units of values
# entries - total number of displayed values
# minDelta - absolute value of the minimum distance between values
#            The maximum distance between values is 2*minDelta
# limits - 'greater', 'less', 'both'
#
def NumericMultipleChoice( prefix, value, ndigs, suffix, entries, minDelta, 
                             limits = 'both' ):

    def fmt_val( val, dig ):
        return "{vl:.{dg}f}".format( vl = val, dg = dig )

    num_err = entries-1
    options = [ [ "true", value ] ]

    if limits == 'greater':
      nup = num_err
      ndn = 0
    elif limits == 'less':
      ndn = num_err
      nup = 0
    else:
      nup = np.random.randint( 1, num_err ) # at least one up/dn
      ndn = num_err - nup
      
    dn_val = value
    for i in range( ndn ):
      dd = np.fabs( minDelta ) * ( 1.0 + np.random.uniform(low=0.0, high=1.0) )
      dn_val -= dd
      options.append( [ "false", dn_val ] )

    up_val = value
    for i in range( nup ):
      # minDelta <= dd <= 2*minDelta
      dd = np.fabs( minDelta ) * ( 1.0 + np.random.uniform(low=0.0, high=1.0) )
      up_val += dd
      options.append( [ "false", up_val ] )

    np.random.shuffle(options)
    
    # set the global variables
    gdic = globals()
    for i, opt in enumerate( options ):
        val = fmt_val( opt[1], ndigs ) + ' ' + suffix
        gdic[ '%s_answer_%i' % (prefix,i) ] = opt[0]
        gdic[ '%s_value_%i'  % (prefix,i) ] = val
        OfflinePrint( '[%s_answer_%i] (%-5s) %s' % ( prefix, i, opt[0], val ) )
    OfflinePrint()

####################################################################
# Shuffles all the answers and generates the global variables 
# associated with
#  $a_answer_0,..., $a_answer_3 and
#  $a_value_0,..., $a_value_3 and
#
#  <multiplechoiceresponse>
#    <choicegroup type="MultipleChoice">
#      <choice correct="$a_answer_0" > $a_value_0 </choice>
#      <choice correct="$a_answer_1" > $a_value_1 </choice>
#      <choice correct="$a_answer_2" > $a_value_2 </choice>
#      <choice correct="$a_answer_3" > $a_value_3 </choice>
#    </choicegroup>
#  </multiplechoiceresponse>
#
#-------------------------------------------------------------------
# Parameters:
#
# prefix - the '*' used in the variables *_answer_0 and *_value_0
# correct: string with the correct answer   
# wrong_lst: list of string with wrong answers
#
def StringMultipleChoice( prefix, shuffle_lst, fixed_lst=None ):

    # shuffle and pick num_err entries
    options = list( shuffle_lst ) 
    np.random.shuffle( options )
    if fixed_lst != None:
      options += list( fixed_lst )
    
    # set the global variables
    gdic = globals()
    for i, opt in enumerate( options ):
        gdic[ '%s_answer_%i' % (prefix,i)  ] = opt[0]
        gdic[ '%s_value_%i'  % (prefix,i)  ] = opt[1]
        OfflinePrint( '[%s_answer_%i] (%-5s) %s\n' % ( prefix, i, opt[0], opt[1] ) )
    OfflinePrint()

####################################################################

In [140]:
InitRandomGenerator( "Question 1--")

In [165]:
####################################################################
# Problem data

Cp_max = 0.49 
lambda_max = 7.5
rho_w = 1025.0

Ucur_lst  = ( 1.5, 2.0, 2.5 )  # [m] 
Prated_lst = ( 0.50E6, 0.75E6, 1.00E6, 1.25E6 ) # [W]

Ucur = RandomPick( Ucur_lst )
Prated = RandomPick( Prated_lst )

Prated_MW = Prated/1E6

Poper = Prated * ( 0.5 + 0.3*np.random.uniform() )
Poper_MW = round( Poper / 1E6, 2 ) 

####################################################################
Radius = np.sqrt( Prated / ( 0.5*rho_w*Ucur**3*np.pi*Cp_max ) )
Diam = 2.0 * Radius

opt = NumericMultipleChoice( prefix='a', value=Diam, ndigs=1, suffix='m', entries=5, minDelta=0.1, limits = 'both' )

####################################################################
Uoper = ( Poper / ( 0.5*rho_w*np.pi*Radius**2*Cp_max ) )**(1.0/3.0)

Omega = lambda_max*Uoper / Radius

NumericMultipleChoice( prefix='b', value=Omega, ndigs=2, suffix='rad/s', entries=5, minDelta=0.2, limits = 'both' )

both 5 3 1
[a_answer_0] (false) 34.1 m
[a_answer_1] (false) 33.9 m
[a_answer_2] (false) 33.4 m
[a_answer_3] (false) 33.8 m
[a_answer_4] (true ) 33.6 m

both 5 2 2
[b_answer_0] (false) 0.87 rad/s
[b_answer_1] (false) 0.02 rad/s
[b_answer_2] (false) 1.25 rad/s
[b_answer_3] (false) 0.26 rad/s
[b_answer_4] (true ) 0.59 rad/s



In [123]:
np.random.randint( 0, 10 )

4

In [99]:
rng = np.random.default_rng(12345)


In [115]:
rng.integers(low=0, high=10)

1